# Simple CoNGA pipeline

This should be roughly equivalent to running the two commands:

**SETUP:**
```
python scripts/setup_10x_for_conga.py --filtered_contig_annotations_csvfile ./conga_datasets/raw/vdj_v1_hs_pbmc_t_filtered_contig_annotations.csv --organism human --output_clones_file tmp_hs_pbmc_clones.tsv
```

**ANALYSIS:**
```
python scripts/run_conga.py --gex_data ./conga_datasets/raw/vdj_v1_hs_pbmc_5gex_filtered_gene_bc_matrices_h5.h5 --gex_data_type 10x_h5 --clones_file tmp_hs_pbmc_clones.tsv --organism human --graph_vs_graph --outfile_prefix tmp_hs_pbmc 
```

In [ ]:
%matplotlib inline

import scanpy as sc
import conga
#import conga.tc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from conga.tcrdist.make_10x_clones_file import make_10x_clones_file

## Input data, output paths

In [ ]:
# you might have to change these paths depending on what you want to analyze
gex_datafile = './conga_datasets/raw/vdj_v1_hs_pbmc_5gex_filtered_gene_bc_matrices_h5.h5'
gex_datatype = '10x_h5' # other possibilities right now: ['10x_mtx', 'h5ad'] (h5ad from scanpy)
tcr_datafile = './conga_datasets/raw/vdj_v1_hs_pbmc_t_filtered_contig_annotations.csv'
organism = 'human'

clones_file = 'tmp_hs_pbmc_clones.tsv'
outfile_prefix = 'tmp_hs_pbmc'


## Setup for conga: make a TCRdist clones_file and compute kernel PCs

In [ ]:
# this creates the TCRdist 'clones file'

make_10x_clones_file( tcr_datafile, organism, clones_file )

# this command will create another file with the kernel PCs for subsequent reading by conga
conga.preprocess.make_tcrdist_kernel_pcs_file_from_clones_file( clones_file, organism )

## Read the data, create a scanpy AnnData object with everything inside

In [ ]:
adata = conga.preprocess.read_dataset(gex_datafile, gex_datatype, clones_file )

# store the organism info in adata
adata.uns['organism'] = organism

adata

Note that the AnnData object contains the TCR sequence information in the `obs` array, stored with the  `['va',..., 'cdr3b_nucseq']` fields.

It also contains the TCRdist kernel PCS in the `obsm` array stored under the key `'X_pca_tcr'`

In [ ]:
# top 50 TCRdist kPCS 
adata.obsm['X_pca_tcr'].shape

In [ ]:
# CDR3-alpha regions:
adata.obs['cdr3a'].head(3)

## Do some very basic scGEX filtering

You could customize this or replace it with something else entirely.

In [ ]:
adata = conga.preprocess.filter_and_scale( adata )

## Now we reduce to a single cell per TCR clonotype

Do this by computing GEX distances and picking the most representative cell for each clonotype (the one with the smallest average distance to all other cells in the clonotype)

In [ ]:
adata = conga.preprocess.reduce_to_single_cell_per_clone( adata )

adata

## Now that we've reduced to a single cell per clonotype, run clustering and dimensionality reduction for GEX and for TCR

In [ ]:
adata = conga.preprocess.cluster_and_tsne_and_umap( adata )


In [ ]:
adata

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
xy = adata.obsm['X_gex_2d']
clusters = np.array(adata.obs['clusters_gex'])
cmap = plt.get_cmap('tab20')
colors = [ cmap.colors[x] for x in clusters]
plt.scatter( xy[:,0], xy[:,1], c=colors)
plt.title('GEX UMAP colored by GEX clusters')

plt.subplot(122)
xy = adata.obsm['X_tcr_2d']
clusters = np.array(adata.obs['clusters_tcr'])
cmap = plt.get_cmap('tab20')
colors = [ cmap.colors[x] for x in clusters]
plt.scatter( xy[:,0], xy[:,1], c=colors)
plt.title('TCR UMAP colored by TCR clusters');



## Compute the GEX and TCR neighbor sets

Also do a bit of other miscellaneous setup

In [ ]:
# these are the nbrhood sizes, as a fraction of the entire dataset:
nbr_fracs = [0.01, 0.1]

# we use this nbrhood size for computing the nndists
nbr_frac_for_nndists = 0.01

all_nbrs, nndists_gex, nndists_tcr = conga.preprocess.calc_nbrs(
    adata, nbr_fracs, also_calc_nndists=True, nbr_frac_for_nndists=nbr_frac_for_nndists)

# stash these in obs array, they are used in a few places...                                                                                                                
adata.obs['nndists_gex'] = nndists_gex
adata.obs['nndists_tcr'] = nndists_tcr

conga.preprocess.setup_tcr_cluster_names(adata) #stores in adata.uns                                                                                                        


## Run graph-vs-graph analysis

In [ ]:
results = conga.correlations.run_graph_vs_graph(adata, all_nbrs)

conga_scores = adata.obs['conga_scores']

good_mask = ( conga_scores <= 1.0 )
adata.obs['good_score_mask'] = good_mask

print(f'found {np.sum(good_mask)} conga hits')

results.sort_values('conga_score', inplace=True)

results.head(3)

### Write the results to a file

In [ ]:
# write the results to a tsv file
clusters_gex = np.array(adata.obs['clusters_gex'])
clusters_tcr = np.array(adata.obs['clusters_tcr'])

indices = results['clone_index']
results['gex_cluster'] = clusters_gex[indices]
results['tcr_cluster'] = clusters_tcr[indices]
for tag in 'va ja cdr3a vb jb cdr3b'.split():
    results[tag] = list(adata.obs[tag][indices])
tsvfile = outfile_prefix+'_graph_vs_graph_hits.tsv'
print('saving graph-vs-graph results to file:',tsvfile)

results.to_csv(tsvfile, sep='\t', index=False)


### Make a scatterplot colored by conga score

In [ ]:
#put the conga hits on top
colors = np.sqrt(np.maximum(-1*np.log10(conga_scores),0.0))
reorder = np.argsort(colors)

plt.figure(figsize=(12,6))
plt.subplot(121)
xy = adata.obsm['X_gex_2d']
plt.scatter( xy[reorder,0], xy[reorder,1], c=colors[reorder], vmin=0, vmax=np.sqrt(5))
plt.title('GEX UMAP colored by conga score')

plt.subplot(122)
xy = adata.obsm['X_tcr_2d']
plt.scatter( xy[reorder,0], xy[reorder,1], c=colors[reorder], vmin=0, vmax=np.sqrt(5))
plt.title('TCR UMAP colored by conga score');


## Make the bicluster logo plots

In [ ]:
nbrs_gex, nbrs_tcr = all_nbrs[0.1]

min_cluster_size = 5

# calc tcr sequence features of good cluster pairs
good_bicluster_tcr_scores = conga.correlations.calc_good_cluster_tcr_features(
    adata, good_mask, clusters_gex, clusters_tcr, conga.tcr_scoring.all_tcr_scorenames, verbose=False,
    min_count=min_cluster_size)

# run rank_genes on most common biclusters
rank_genes_uns_tag = 'rank_genes_good_biclusters'
conga.correlations.run_rank_genes_on_good_biclusters(
    adata, good_mask, clusters_gex, clusters_tcr, min_count=min_cluster_size, key_added= rank_genes_uns_tag)

gex_header_tcr_score_names = ['mhci2', 'cdr3len', 'cd8', 'nndists_tcr']

logo_pngfile = outfile_prefix+'_bicluster_logos.png'

conga.plotting.make_logo_plots(
    adata, nbrs_gex, nbrs_tcr, min_cluster_size, logo_pngfile,
    good_bicluster_tcr_scores=good_bicluster_tcr_scores,
    rank_genes_uns_tag = rank_genes_uns_tag,
    gex_header_tcr_score_names = gex_header_tcr_score_names )


### The image that was saved to disk is higher resolution than the one above here in the notebook 
This should be the one on disk, below. You could try opening it in a new tab, to see the full resolution...
![hey](tmp_hs_pbmc_bicluster_logos.png)

## Run graph-vs-feature analysis, comparing the TCR graph to GEX features (mostly gene expression)

In [ ]:
pval_threshold = 1.
results = []
for nbr_frac in nbr_fracs:
    nbrs_gex, nbrs_tcr = all_nbrs[nbr_frac]
    print('finding biased GEX features for nbrhoods with size', nbr_frac, nbrs_gex.shape)
    results.append( conga.correlations.tcr_nbrhood_rank_genes_fast( adata, nbrs_tcr, pval_threshold, verbose=False))
    results[-1]['nbr_frac'] = nbr_frac

# save the results to a file
tsvfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features.tsv'
print('making:', tsvfile)
results_df = pd.concat(results, ignore_index=True)
results_df.to_csv(tsvfile, index=False, sep='\t')

pngfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features.png'
print('making:', pngfile)
conga.plotting.plot_ranked_strings_on_cells(
    adata, results_df, 'X_tcr_2d', 'clone_index', 'mwu_pvalue_adj', 1.0, 'feature', pngfile)

### Plot the top GEX features on the TCR UMAP landscape

In [ ]:
pngfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features_panels.png'
print('making:', pngfile)
conga.plotting.make_feature_panel_plots(adata, 'tcr', all_nbrs, results_df, pngfile)

## Run graph-vs-features analysis, comparing the GEX graph to TCR features

In [ ]:
pval_threshold = 1.
results = []
tcr_score_names = conga.tcr_scoring.all_tcr_scorenames # the TCR features to use
for nbr_frac in nbr_fracs:
    nbrs_gex, nbrs_tcr = all_nbrs[nbr_frac]
    results.append( conga.correlations.gex_nbrhood_rank_tcr_scores(
        adata, nbrs_gex, tcr_score_names, pval_threshold, verbose=False ))
    results[-1]['nbr_frac'] = nbr_frac
results_df = pd.concat(results, ignore_index=True)

tsvfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features.tsv'
print('making:', tsvfile)
results_df.to_csv(tsvfile, index=False, sep='\t')

pngfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features.png'
print('making:', pngfile)

conga.plotting.plot_ranked_strings_on_cells(
    adata, results_df, 'X_gex_2d', 'clone_index', 'mwu_pvalue_adj', 1.0, 'feature', pngfile,
    direction_column='ttest_stat')

### Plot the top TCR score hits on the GEX UMAP landscape

In [ ]:
pngfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features_panels.png'
print('making:', pngfile)
conga.plotting.make_feature_panel_plots(adata, 'gex', all_nbrs, results_df, pngfile)